# 2 - Agentic RAG with Bedrock KB and LlamaIndex SubQuestionQueryEngine

Naive RAG based on top-k search often fail to address complex queries that require reasoning about multiple entities.

In this notebook, we build an agentic RAG system with Bedrock KB and LlamaIndex `SubQuestionQuery Engine`.

Additional resources on `SubQuestionQueryEngine`:
* tutorial: https://docs.llamaindex.ai/en/stable/examples/query_engine/sub_question_query_engine/
* API docs: https://docs.llamaindex.ai/en/stable/api_reference/query_engine/sub_question/

### Installation

In [2]:
%pip install llama-index
%pip install llama-index-llms-bedrock
%pip install llama-index-embeddings-bedrock
%pip install llama-index-retrievers-bedrock


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Pre-req: Created Amazon Bedrock KB

Here, you should have a running Amazon Bedrock KB. To create one, either:
1. via the AWS managment console, or
2. programmatically following this guide: https://github.com/aws-samples/amazon-bedrock-workshop/blob/main/02_KnowledgeBases_and_RAG/0_create_ingest_documents_test_kb.ipynb

Note down the knowledge base ID.

In [31]:
knowledge_base_id = "JQXQKUAFMZ"

### Setup and imports

In [40]:
from llama_index.core import Settings
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.response.pprint_utils import pprint_response

from llama_index.retrievers.bedrock import AmazonKnowledgeBasesRetriever
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding

In [5]:
llm = Bedrock(model = "anthropic.claude-v2")
embed_model = BedrockEmbedding(model = "amazon.titan-embed-text-v1")

In [6]:
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 1024

### Setup agentic RAG with SubQuestionQueryEngine

We setup separate retriever & query engine for Uber and Lyft via metadata filters.

In [32]:
top_k = 4
search_mode = "HYBRID"

In [33]:
lyft_fname = "lyft_2021.pdf"
lyft_retriever = AmazonKnowledgeBasesRetriever(
    knowledge_base_id=knowledge_base_id,
    retrieval_config={
        "vectorSearchConfiguration": {
            "numberOfResults": top_k,
            "overrideSearchType": search_mode,
            "filter": {"equals": {"key": "file_name", "value": lyft_fname}},
        }
    },
)
lyft_engine = RetrieverQueryEngine(retriever=lyft_retriever)

In [34]:
uber_fname = "uber_2021.pdf"
uber_retriever = AmazonKnowledgeBasesRetriever(
    knowledge_base_id=knowledge_base_id,
    retrieval_config={
        "vectorSearchConfiguration": {
            "numberOfResults": top_k,
            "overrideSearchType": search_mode,
            "filter": {"equals": {"key": "file_name", "value": uber_fname}},
        }
    },
)
uber_engine = RetrieverQueryEngine(retriever=uber_retriever)

In [35]:
tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_2021",
            description="10K filing for Lyft 2021",
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_2021",
            description="10K filing for Uber 2021",
        ),
    ),
]

In [36]:
from llama_index.core.query_engine import SubQuestionQueryEngine

In [37]:
query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=tools,
)

### Run queries

In [38]:
response = await query_engine.aquery('Compare revenue growth of Uber and Lyft from 2020 to 2021')

Generated 2 sub questions.
[uber_2021] Q: What was Uber's revenue growth from 2020 to 2021?
[uber_2021] A: Based on the context provided, Uber's revenue grew 57% from 2020 to 2021. Specifically, the context states:

Revenue increased $6.3 billion, or 57%, primarily attributable to an increase in Gross Bookings of 56%, or 53% on a constant currency basis.

And later: 

Revenue was $17.5 billion, or up 57% year-over-year, reflecting the overall growth in our Delivery business and an increase in Freight revenue attributable to the acquisition of Transplace in the fourth quarter of 2021 as well as growth in the number of shippers and carriers on the network combined with an increase in volumes with our top shippers.

So Uber's revenue grew by 57% from 2020 to 2021.
[lyft_2021] Q: What was Lyft's revenue growth from 2020 to 2021?
[lyft_2021] A: Based on the provided context, Lyft's revenue in 2021 was $3.2 billion, an increase of 36% compared to revenue of $2.4 billion in 2020. Specifically

In [41]:
pprint_response(response, show_source=True)

Final Response: Based on the provided context, Uber's revenue grew by
57% from 2020 to 2021, increasing from $11.1 billion to $17.5 billion.
Lyft's revenue grew by 36% over the same period, increasing from $2.4
billion in 2020 to $3.2 billion in 2021.   In summary, Uber
experienced higher revenue growth compared to Lyft from 2020 to 2021,
with Uber's revenue growing by 57% compared to 36% growth for Lyft.
However, both companies saw strong revenue growth during this period
as demand recovered from the COVID-19 pandemic.
______________________________________________________________________
Source Node 1/10
Node ID: ae3edb68-3448-43c9-a502-4824137c3112
Similarity: None
Text: Sub question: What was Uber's revenue growth from 2020 to 2021?
Response: Based on the context provided, Uber's revenue grew 57% from
2020 to 2021. Specifically, the context states:  Revenue increased
$6.3 billion, or 57%, primarily attributable to an increase in Gross
Bookings of 56%, or 53% on a constant currency 

In [42]:
response = await query_engine.aquery('Compare the investments made by Uber and Lyft')

Generated 2 sub questions.
[uber_2021] Q: What were the major investments made by Uber in 2021?
[uber_2021] A: Based on the provided context, in 2021 Uber:

- Divested its JUMP business and contributed JUMP assets to Lime in exchange for common stock, preferred stock, and warrants in Lime. This gave Uber an approximately 10% fully diluted ownership interest in Lime.

- Made investments in and partnerships with platforms like Carmera and Motional to advance autonomous vehicle technology. 

- Continued expanding its Delivery business globally, which grew Gross Bookings by 66% in 2021.

- Saw increased demand for its Mobility business as Trip volumes recovered from the impacts of COVID-19, with Mobility Gross Bookings growing 36% in 2021.

The context indicates these were some of the major investments and strategic moves for Uber in 2021 as it continued to expand its businesses globally.
[lyft_2021] Q: What were the major investments made by Lyft in 2021?
[lyft_2021] A: Based on the conte

In [44]:
pprint_response(response, show_source=True)

Final Response: Based on the provided context, here is a comparison of
the major investments made by Uber and Lyft in 2021:  Uber: - Divested
its JUMP business and gained equity stake in Lime through the deal. -
Made investments in autonomous vehicle technology companies like
Carmera and Motional.   - Expanded its Delivery business globally with
66% Gross Bookings growth. - Saw increased demand for Mobility
business with 36% Gross Bookings growth.  Lyft: - Invested in research
and development to launch new innovations. - Made strategic
acquisitions to complement their business (unspecified companies). -
Invested in sales, marketing and brand building. - Continued investing
for future growth despite COVID-19 impacts.   - Expanded network of
Light Vehicles and Lyft Autonomous for self-driving tech. - Invested
in LyftUp for affordable transportation options. - Committed to 100%
electric vehicles by 2030.  In summary, both made significant
investments in 2021 - Uber in divesting assets and